<a href="https://colab.research.google.com/github/suzukimain/suzukimain_image_generation/blob/main/Image_generation_ver_Bete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#重要な注意点(Important Notes)：
 **・商用利用はご遠慮ください。**

 (Please do not use for commercial purposes.)

**・画像生成によって起こった問題について、私は一切責任を負いません。**

(I am not responsible for any problems caused by the image generation.)

**・使用するにあたってはくれぐれも、自己責任でお願いします**

(Please use at your own risk.)

上記の内容と、最下部に記載されている利用規約に同意できない方は、本画像生成ブックの使用はご遠慮ください。

(If you do not agree to the above contents and the terms of use listed at the bottom, please do not use this image generation book.)

追記:
本ノートブックを使用した時点で重要な注意点と、利用規約に同意したとみなします。

(P.S. By using this notebook, you agree to the important notes and terms of use.)

#1.ランタイムをGPUにしてください 
(1. Please set the runtime to GPU.).          

ランタイム>ランタイムのタイプを変更>Noneを押す>GPUを選択>決定.                     
(Runtime > Change runtime type > Select 'GPU' after clicking 'None' > Save)

In [ ]:
#-----------------------------------------------------------------------------
#これは画像生成モデルです
#-----------------------------------------------------------------------------

In [ ]:
#テキストのみよりも、画像とセットで設定する方が精度は高いです。

In [ ]:
#このノートブックは開発中ですので、エラーが起きる可能性があります。

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
#@title  { run: "auto", display-mode: "code"}
# @markdown # 2.このコードを実行してください (2. Please run this code.)
# @markdown 左の三角形が書いてあるボタンを押してください (Press the button with the triangle on the left.)
# @markdown >使用するモデルの切り替え (model change)

model_select = "Reality model" #@param ["Reality model", "Anime model"]

# @markdown Reality model**：**現実に近い画像を生成するのに適しています

#@markdown Anime model**：**アニメの画像を生成するのに適しています


# @markdown >モードの切り替え (mode change)
mode_select = "Nomal" #@param ["Quick", "Nomal"]
# @markdown 生成時間が長くなるけど、より奇麗になります

入力形式 = "image_and_text" #@param ["text_only", "image_and_text"]

# @markdown

Googleドライブに接続 = True  # @param {type:"boolean"} 
if Googleドライブに接続: 
    from google.colab import drive 
    drive.mount('/content/drive') 
# @markdown  **チェックをつけると接続に必要な権限を求めるようになります** 

# @markdown (Checking this box will prompt you for the necessary permissions to connect.)

# @markdown


# @markdown ※モード切り替えをしたら再度実行して下さい


# @markdown (It is recommended to use this service while connected to Wi-Fi.)


#@markdown インストールする手順を飛ばします

try:
    import torch
    import diffusers
    import transformers
    import accelerate
    import scipy
    import safetensors
    import ftfy
    import regex
    import tqdm
    import sentencepiece
    import pysbd


except ImportError:
    !pip install torch==2.0.1+cu118 diffusers==0.16.1 transformers==4.29.2 accelerate==0.19.0 scipy==1.10.1 safetensors==0.3.1 ftfy==6.1.1 regex==2022.10.31 tqdm==4.65.0 scipy==1.10.1 sentencepiece==0.1.99 pysbd==0.3.4  -q
    
    import torch
    import diffusers
    import transformers
    import accelerate
    import scipy
    import safetensors
    import ftfy
    import regex
    import tqdm
    import sentencepiece
    import pysbd

try:
 import clip
except ModuleNotFoundError:
  !pip install git+https://github.com/openai/CLIP.git -q



#インストールをスキップ = False  # @param {type:"boolean"} 
モデルをドライブに保存する = False  # @param {type:"boolean"}
ドライブに保存したモデルを使う = False  # @param {type:"boolean"}


#if not インストールをスキップ:
 # !pip install git+https://github.com/Stability-AI/stablediffusion.git -q


import os
import spacy
import clip
import codecs
import pickle
import torch
import random
import numpy as np
import transformers
import sentencepiece
import ipywidgets as widgets 
from PIL import Image
#from stegano import lsb
from datetime import datetime 
from transformers import pipeline
from torch import Generator, autocast
from IPython.display import display, Markdown 
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from diffusers import EulerDiscreteScheduler, StableDiffusionImg2ImgPipeline
#from diffusion_script_util import ModelAndLoss
#from diffusion import utils
from transformers import GPT2Tokenizer, GPTNeoForCausalLM
from transformers import pipeline, GPTNeoForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 使用するモデルを設定
if model_select == "Reality model":
   model_id = "stabilityai/stable-diffusion-2-1"
else:
   model_id = "hakurei/waifu-diffusion"

# モードの切り替え (mode change)
dtype = torch.float16 if mode_select == "Quick" else torch.float32

diffuser_kwargs = {
   "negative_prompt": "worst quality, low quality, medium quality, deleted, lowres, comic, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry",
   "seed": 72566,
   "temperature": 0.8,
   "truncation": 0.8,
   "diffusion_steps": 500
}

if ドライブに保存したモデルを使う:
    if model_select == "Anime model":
        model_path = "/content/drive/MyDrive/image_generation/model/waifu_diffusion/waifu_diffusion_model/model.pt"
        pipeline_path = "/content/drive/MyDrive/image_generation/model/waifu_diffusion/waifu_diffusion_pipeline/pipeline.pt"
        #model_id = "stabilityai/stable-diffusion-2-1"
        model_state_dict = torch.load(model_path)
        #model = model_id(state_dict=model_state_dict)
        pipe = torch.load(pipeline_path)
        #pipe.model = model  
    elif model_select == "Reality model":
        # Realityモデルの.ptファイルへのパス
        model_path = "/content/drive/MyDrive/image_generation/model/stable_diffusion/stable_diffusion_model/model.pt"
        pipeline_path = "/content/drive/MyDrive/image_generation/model/stable_diffusion/stable_diffusion_pipeline/pipeline.pt"
        #model_id = "hakurei/waifu-diffusion"
        model_state_dict = torch.load(model_path)
        #model = model_id(state_dict=model_state_dict)
        pipe = torch.load(pipeline_path)
        #pipe.model = model
    else:
        print("申し訳ありません。現時点で実装されていないモデルになります")


# パーツ単位で読み込み、再構築します。
    
else:
    model_path = None
    pipeline_path = None
    if 入力形式 == "text_only":
        # StableDiffusion(メインのモジュール)パイプライン設定 
        # もしも何かあったら"torch_dtype=torch.float16"をつける
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id, torch_dtype=dtype,
            clip_model=lambda x: model.encode_image(x).float(),
        )
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

    else:
        #StableDiffusionパイプライン設定
        scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)


# 使用する計算機を設定(GPUがない場合は"cpu"に変更) 
pipe = pipe.to(device)

if モデルをドライブに保存する:
    # 使用するモデルを選択
    if model_select == "Reality model":
        model_id = "stabilityai/stable-diffusion-2-1"
        save_model_dir = "/content/drive/MyDrive/image_generation/model/stable_diffusion/stable_diffusion_model"
        save_pipeline_dir = "/content/drive/MyDrive/image_generation/model/stable_diffusion/stable_diffusion_pipeline"
    else:
        model_id = "hakurei/waifu-diffusion"
        save_model_dir = "/content/drive/MyDrive/image_generation/model/waifu_diffusion/waifu_diffusion_model"
        save_pipeline_dir = "/content/drive/MyDrive/image_generation/model/waifu_diffusion/waifu_diffusion_pipeline"

    model_save_path = os.path.join(save_model_dir, "model.pt")
    pipeline_save_path = os.path.join(save_pipeline_dir, "pipeline.pt")

    if not os.path.exists(save_model_dir):
        os.makedirs(save_model_dir, exist_ok=True)  # フォルダが存在しない場合は作成
        torch.save(pipe.model.state_dict(), model_save_path)
    else:
        print("既にモデルはアップロードされているため、保存をスキップしました。")
    if not os.path.exists(save_pipeline_dir):
        os.makedirs(save_pipeline_dir, exist_ok=True)  # フォルダが存在しない場合は作成
        torch.save(pipe, pipeline_save_path) 
    else:
        print("既にパイプラインはアップロードされているため、保存をスキップしました。")


#\033[31mが赤、\033[33mが黄色、\033[34mが青、\033[32mが緑、\033[0mが白
# "\033[32m" は緑色に変更するための"ANSI Escape Code"であり、"\033[0m"はデフォルトの文字色に戻すためのコードです。
print("\033[32m画像生成の準備が出来たので、手順3に移ってください。\033[0m")   
print("\033[32m(Now that the image generation is ready, please proceed to step 3.)\033[0m")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
画像生成の準備が出来たので、手順3に移ってください。
(Now that the image generation is ready, please proceed to step 3.)


In [ ]:
#@markdown #3.このコードを実行してください(3. Please run this code.)
# @markdown >生成したい枚数を入力してください　(Please enter the number of images you want to generate here.)
生成する枚数 = 20 #@param {type:"slider", min:1, max:100, step:1}
#if 生成する枚数 <= 0 or isinstance(生成する枚数, float): 
  #  print("\033[31m警告:無効な数字が入力された為デフォルトの1枚に設定しました\033[0m") 
  #  生成する枚数 = 1 
#if 生成する枚数 is None: 
 #   print("\033[31m警告:無効な形式な為デフォルトの1枚に設定しました\033[0m") 
#    生成する枚数 = 1 

# @markdown  デフォルトでは1回につき1枚生成します。(By default, one image will be generated per run.)

# @markdown ------

# @markdown  >生成する画像の条件を**英語で**入力してください　(Please input the conditions for generating images in **English**.)

Prompt = "cute, cat" #@param {type:"string"}
画像の質を上げるプロントを追加する = True #@param {type:"boolean"}
日本語入力 = False #@param {type:"boolean"}
# @markdown 例、"Earth, space, high resolution" や "Please draw a beautiful Mount Fuji with the sun rising from the summit."、"cute, cat" 

# @markdown ------

# @markdown >画像を入力として送リます。

# @markdown 入力形式を "image_and_text" とした場合のみに使えます 


入力する画像 = "" #@param {type:"string"}


# @markdown ------

seed値 = None #@param {type:"number"}
guidance_scale = 25 #@param {type:"slider", min:1, max:40, step:1}
温度 = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
#top_k = 17 #@param {type:"slider", min:1, max:40, step:1}
#top_p = 0 #@param {type:"slider", min:0, max:1, step:0.01}
雰囲気 = 0.7 #@param {type:"slider", min:0.1, max:1, step:0.1}
拡散ステップ = 50  # @param {type:"number"}
#safe_level = "safe" #@param ["safe", "Questionable", "Explicit"]
縦の解像度 = "1080" #@param ["480","512", "768", "1080","1152", "1440", "1920", "3840", "7680"]
横の解像度 = "1080" #@param ["480","512", "768", "1080","1152", "1440", "1920", "3840", "7680"]
height = int(縦の解像度)
width = int(横の解像度)

#@markdown seed(0≦**∞**)=seed値を指定します。何も入力しない・Noneの場合はランダムです。

#@markdown guidance_scale(1≦40 推奨"25")どのくらい細かく描くかを指定します。

#@markdown 温度(0.1≦1.0 推奨0.8)＝どのくらい条件に合わせるかを指定します。

#@markdown 拡散ステップ(1≦1000 推奨400)＝何回計算をするかです。回数を減らすほど生成速度が速くなります

#@markdown 解像度(推奨1080)=解像度を指定します。縦横が異なっていても問題ありません。
#@markdown 数値を小さくしても、あまり生成速度に影響はありません

# @markdown ------

# @markdown >必要であればネガティブプロンプトを入力してください。人物を生成するときにおすすめです

# @markdown よくわからない方ボタンを押してください。おすすめのネガティブプロントを使います

# @markdown (If necessary, please enter a negative prompt. It is recommended for generating characters.)

N_prompt = ""  # @param {type:"string"}
人物生成におすすめのネガティブプロントを使う = True #@param {type:"boolean"}
# @markdown ネガティブプロンプトとは、**ネガティブな要素を除く**ものです。 例、low quality、unbeautiful.

# @markdown (Negative prompts are used to exclude negative elements from an image. For example, you can use a negative prompt to exclude low quality images or images that are not beautiful.)

# @markdown ------

#@markdown >保存する先を指定します

# @markdown (If you would like to specify a location to save the generated image, please enter the path.)

保存する先のパス = ""  # @param {type:"string"} 
# @markdown  入力しなければ "/content/生成した画像" に保存されます。なければ作るようになっています。

#@markdown (If no path is specified, the images will be saved to "/content/生成した画像". The directory will be created if it doesn't exist.)

# @markdown

# @markdown 補足."/content/生成した画像"は左のタスクバーにあるファイルのマークを押すと"生成した画像"のフォルダを見れます
 
# @markdown (P.S.You can see the images generated at '/content/生成した画像' by clicking the file icon in the left taskbar.)

# @markdown ------

# @markdown #  オプション(Options) 

# @markdown >フィルターを調整

#@markdown 注意事項に同意して頂けたら、下のボックスにチェックをしたのち、 "filter_off" と入力をお願いします。

#@markdown **注意事項:不適切な画像が生成された時そのまま表示される可能性があります。変更にあたっては自己責任でお願いします**


上記のことに同意します = False  # @param {type:"boolean"} 

filter_checker = "" #@param {type:"string"}

if 上記のことに同意します:
  if filter_checker=="filter_off":
    pipe.safety_checker = None
  else:
    pipe.safety_checker = False
else:
  pipe.safety_checker = False



#@markdown 変更によって、不適切な画像がそのまま出てくる可能性があります.


#@markdown 

# @markdown > 生成した画像をGoogleドライブに保存したいときにチェックを入れてください             

# @markdown (Check this box to save the generated image to Google Drive.)

# @markdown

# @markdown #**エラーが起きたら1.を再実行して下さい**
 
# @markdown #(If an error occurs, please try step 1 again.)

# @markdown (なおこの画像生成は "stable diffusion-ver.2" を使用しています)

# @markdown (Please note that this image generation uses "stable diffusion-ver.2".)

# @markdown 生成した画像を画面に表示します

画面に表示 = False  # @param {type:"boolean"} 


# ユーザーが入力した文字列を取得する

if 入力形式=="image_and_text":
  init_image = Image.open(入力する画像)
  init_image = init_image.convert("RGB")
  init_image = init_image.resize((768, 512))


if 日本語入力:
  texts = Prompt
  encoded_text = codecs.encode(texts, 'utf-8')
  decoded_text = codecs.decode(encoded_text, 'utf-8')
  pos_tagger = spacy.load('en_core_web_sm')
  fugu_translator = pipeline('translation', model='staka/fugumt-ja-en')
  translation = fugu_translator(decoded_text, src_lang="ja", tgt_lang="en")[0]["translation_text"]
  After_Prompt = translation
else:
  After_Prompt = Prompt


if not Prompt:
   print("\033[1m\033[31m条件の入力がないため一時停止します\033[0m")
   Prompt = input("入力した後Enterを押すと再開します。")

if seed値 is None:
   seed値 = random.randint(1,4294967295)#32bitの最大値
   
if 画像の質を上げるプロントを追加する:
  追加 = "((masterpiece)), best quality, high quality, absurdres"
else:
  追加 = ""  

if 人物生成におすすめのネガティブプロントを使う:
  おすすめのネガティブプロント = "((((worst quality)))), ((((low quality)))), blurred, medium quality, purple hair, deleted, lowres, comic, bad anatomy, (((bad hands))), low resolution, blurry, pixelated, sketchy, nonsensical anatomy, unrealistic pose, mosaic, unclear details, distorted colors, unrealistic proportions, poor quality, fuzzy, out of focus, hazy, grainy, text, error, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, standard quality, bad feet_hand_finger_leg_eye, bad, text font ui, bad shadow, poorly drawn, black-white, ugly, duplicate, mutation, mutilated, malformed mutated, malformed, The background is incoherent, simple background, low-quality background, low background, bad body, long body, broken limb, anatomical nonsense, extra limbs, missing limb, incorrect limb, multiple heads, twisted head, poorly drawn face, head_wings, More than two cows, missing head, 1 unit with multiple heads, heads together, abnormal eye proportion, unclear eyes, bad eyes, fused eyes, poorly drawn eyes, bad mouth, poorly drawn mouth, bad tongue, too long tongue, bad ears, poorly drawn ears, extra ears, heavy ears, long neck, too thick neck, bad neck, abnormally large breasts, extra_breasts, fused breasts, bad breasts, missing arms, disappearing arms, extra arms, three arms, mutated hands and fingers, black hands, fused hand, missing fingers, extra digits, huge thighs, disappearing thigh, missing thighs, extra thighs, bad feet, bad legs, huge calf, disappearing legs, bad gloves, fused gloves, beard, artist name, text watermark, unnatural, obviously wrong, distorted face, floating hair, floating body parts, severed body parts, incorrect leg position, deformed, fused body and hands, disregard of physics, distorted shape, doll-like object not present in the image, body fusion, abnormal fingers, fingers resembling fish fins, dot eyes, unclear background, mosaic, body bending, incorrect leg-to-torso ratio, excessively large breasts, more than three limbs (hands/legs), unsettling appearance, eyes filled with solid color, lack of lower body, splitting, creepy doll-like appearance, distorted eyes, lines on the skin, legs bending in unnatural directions, abnormal finger count, missing arms, floating hands, lack of nose or mouth, hands emerging from impossible places, incorrect body part ratios, bad, longbody, lowres, bad anatomy, bad hands, missing fingers, fat, big boobs, pubic hair, Distorted eye contour, Missing part from the ankles onward, extra digit, fewer digits, split wings, Vampire wings floating in the air, bad wing, cropped"
else:
  おすすめのネガティブプロント =""


NOT_prompt = N_prompt + おすすめのネガティブプロント

negative_prompt2=("(nsfw)," + NOT_prompt)


new_diffuser_kwargs = {
   "negative_prompt":negative_prompt2,
   "seed": seed値,
   "temperature": 温度,
   "truncation": 雰囲気,
   "diffusion_steps": 拡散ステップ
   }

#returnを使うとfor文が機能しないからdef関数は使えない
def generate_images():
    prompt = 追加 + After_Prompt 
    
    seed = seed値
    #generator.manual_seed(seed)
 
    diffuser_kwargs.update(new_diffuser_kwargs) 
    num_images = 生成する枚数
    for i in range(num_images):
       if 入力形式 == "image_and_text":
          with autocast("cuda"):
                result = pipe(prompt, image=init_image, negative_prompt=negative_prompt2, guidance_scale=guidance_scale)
                make_image = result.images[0]  # 生成された画像はリストの最初の要素に格納されている
                if 保存する先のパス:
                  seed=seed値
                  date = datetime.now().strftime("%Y%m%d_%H%M%S")
                  filenames = f"seed({seed})_prompt[{prompt}]_{date}.png" # seed値を含めたファイル名を作成する
                  path = os.path.join(保存する先のパス, filenames)        
                  if not os.path.exists(保存する先のパス):
                     os.makedirs(保存する先のパス)
                  make_image.save(path)
                  print("\033[34m画像生成が完了しました")
                  print("保存先のパス:(\033[32m "+path+"\033[34m)")
                  print("ファイルの名前:(\033[32m"+filenames+"\033[34m)")
                  print("上記のようになります。ご確認ください\033[0m)") 
                else:
                  seed=seed値
                  date = datetime.now().strftime("%Y%m%d_%H%M%S")
                  filenames = f"seed({seed})_prompt[{prompt}]_{date}.png" # seed値を含めたファイル名を作成する
                  path = os.path.join('/content/生成した画像', filenames)
                  if not os.path.exists('/content/生成した画像'):
                     os.makedirs('/content/生成した画像')       
                  make_image.save(path)        
                  print("\033[34m画像生成が完了しました。")
                  print("保存先のパス:(\033[32m content/生成した画像\033[34m)")
                  print("ファイルの名前:(\033[32m"+ filenames +"\033[34m)")
                  print("上記のようになります。ご確認ください\033[0m)")  
                  if 画面に表示:
                     make_image.show()

       else:
          with autocast("cuda"):
          # ユーザーからの最新入力を使用して画像生成(字下げに注意1文字あける) 
                result = pipe(prompt, negative_prompt=negative_prompt2, guidance_scale=guidance_scale,) 
                make_image = result.images[0]              
                if 保存する先のパス:
                  seed=seed値
                  date = datetime.now().strftime("%Y%m%d_%H%M%S")
                  filenames = f"seed({seed})_prompt[{prompt}]_{date}.png" # seed値を含めたファイル名を作成する
                  path = os.path.join(保存する先のパス, filenames)        
                  if not os.path.exists(保存する先のパス):
                     os.makedirs(保存する先のパス)
                  make_image.save(path)
                  print("\033[34m画像生成が完了しました")
                  print("保存先のパス:(\033[32m "+path+"\033[34m)")
                  print("ファイルの名前:(\033[32m"+filenames+"\033[34m)")
                  print("上記のようになります。ご確認ください\033[0m)") 
                else:
                  seed=seed値
                  date = datetime.now().strftime("%Y%m%d_%H%M%S")
                  filenames = f"seed({seed})_prompt[{prompt}]_{date}.png" # seed値を含めたファイル名を作成する
                  path = os.path.join('/content/生成した画像', filenames)
                  if not os.path.exists('/content/生成した画像'):
                     os.makedirs('/content/生成した画像')       
                  make_image.save(path)        
                  print("\033[34m画像生成が完了しました。")
                  print("保存先のパス:(\033[32m content/生成した画像\033[34m)")
                  print("ファイルの名前:(\033[32m"+ filenames +"\033[34m)")
                  print("上記のようになります。ご確認ください\033[0m)")  
                  if 画面に表示:
                      make_image.show()

#threshold = filter_level  # 閾値を適宜調整
generate_images()
#filtered_image = nsfw_filter(image, threshold)

バージョン履歴

2023.5.25 Animeモデルを使用して画像生成をした時に条件の変更が反映されないバグが発生しました

2023.5.26 バグを修正
          画像をプロントとして使えるようになりました。

>説明

フィルターの仕組みは、画像の不適切さを評価するために、0から1の範囲で値を割り当てます。値が0に近いほど不適切な画像の可能性が減ります。また、一般的には0.7以上のスコアと判定されたものは露骨な表現があるとされています。

#**Readme**(日本語版)
#利用規約

本画像生成ノーブックを使用するにあたって、利用規約の内容に全て同意したとみなします。

重要な注意点：
**商用利用はご遠慮ください。**
**画像生成によって起こった問題について、私は一切責任を負いません。**

免責事項:
*  使用にあたっては、自己責任でお願いします。
*  本モデルは予告なく変更・非公開・削除する可能性があります。
*  利用規約は予告なく変更する場合があります。
*  このモデルは、趣味で作成したものであり、商用利用などは意図していません。
*  使用にあたって発生した通信量、電気料金など金銭に関わるものの負担は追い兼ねます
*  Stable Diffusion-Ver2.1やその他の追加ライブラリに関する規約がある場合は、それらも確認することを強くお勧めします。
*  本プロジェクトを利用することにより生じた一切の問題について、私は一切責任を負いません。

ー本プロジェクトとは、本画像生成ノートブックや、githubのページなどをさします
___
#本プロジェクトの説明
Stable Diffusion-Ver2.1をベースにした画像生成ノートブックです。

**使用させていただいたツール**
※2023/5/12時点

-Stable Diffusion-ver2.1

-CLIP

使用させていただいたライブラリ
- Stable Diffusion-ver2.1
- CLIP
- tqdm
- ftfy
- scipy
- regex
- torch 
- diffusers 
- accelerate
- safetensors
- transformers
  
---

謝辞

本画像生成ノートブックの作成にあたり、オープンソースのリソースやフリーのツールを使用させていただきました。個人的な利用でしたが、これらのリソースやツールがあったからこそ、本プロジェクトを実現することができました。
この場を借りて、オープンソースのコミュニティや、フリーのツールを提供してくださった方々に感謝の意を表します。素晴らしいツールや技術を提供してくださり、本プロジェクトを支援してくださったことに心から感謝いたします。

#**Readme(English_ver)**
# Terms of Use

By using this image generation notebook, you agree to all the contents of the Terms of Use.

Important Notice:
**Please refrain from using for commercial purposes.**
**I am not responsible for any problems caused by image generation.**

Disclaimer:
* Please use it at your own risk.
* This model may be changed, unpublished, or deleted without notice.
* The terms of use may be changed without notice.
* This model is created for personal use and is not intended for commercial use.
* I will not be responsible for any financial costs incurred, such as communication costs and electricity costs, incurred during use.
* If there are terms and conditions for Stable Diffusion-Ver2.1 and other additional libraries, it is strongly recommended to confirm them as well.
* I am not responsible for any problems caused by using this project.

ー This project refers to the image generation notebook and GitHub pages.

---

# Description of this project

This is an image generation notebook based on Stable Diffusion-Ver2.1.

**Tools used**
* As of 2023/5/12

- Stable Diffusion-ver2.1
- CLIP

**Libraries used**
- Stable Diffusion-ver2.1
- CLIP
- tqdm
- ftfy
- scipy
- regex
- torch 
- diffusers 
- accelerate
- safetensors
- transformers

---

# Acknowledgements

I used open source resources and free tools to create this image generation notebook. Although it was for personal use, it was only possible to realize this project because of these resources and tools.
I would like to express my gratitude to the open source community and those who provide free tools. I sincerely appreciate your support for this project by providing great tools and technologies. 


うまく行ったseed値

*   2192424118
*   





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
